# 5.11 残差网络（ResNet）

In [6]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

1.9.0
cpu


## 5.11.2 残差块

In [78]:
class Residual(nn.Module):  # 本类已保存在d2lzh_pytorch包中方便以后使用
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1) # stride默认值是1，保持输出形状不变
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, X):
        print(f"X - {X.shape}")
        conv1_out = self.conv1(X)
#         print(f"conv1_out - {conv1_out.shape}")
        bn1_out = self.bn1(conv1_out)
#         print(f"bn1_out - {bn1_out.shape}")
        relu1_out = F.relu(bn1_out)
#         print(f"relu1_out - {relu1_out.shape}")

        conv2_out = self.conv2(relu1_out)
#         print(f"conv2_out - {conv2_out.shape}")
        bn2_out = self.bn2(conv2_out)
#         print(f"bn2_out - {bn2_out.shape}")
        
        if self.conv3:
            X = self.conv3(X)
#             print(f"conv3_out - {X.shape}")
        
        Y = F.relu(bn2_out + X)
        print(f"Y - {Y.shape}")
        return Y

In [69]:
X = torch.rand((4, 3, 6, 6))

In [62]:
blk = Residual(3, 3, use_1x1conv=False, stride=1)
out = blk(X)

X - torch.Size([4, 3, 6, 6])
conv1_out - torch.Size([4, 3, 6, 6])
bn1_out - torch.Size([4, 3, 6, 6])
relu1_out - torch.Size([4, 3, 6, 6])
conv2_out - torch.Size([4, 3, 6, 6])
bn2_out - torch.Size([4, 3, 6, 6])
Y - torch.Size([4, 3, 6, 6])


In [65]:
blk = Residual(3, 6, use_1x1conv=True, stride=2)
out = blk(X)

X - torch.Size([4, 3, 6, 6])
conv1_out - torch.Size([4, 6, 3, 3])
bn1_out - torch.Size([4, 6, 3, 3])
relu1_out - torch.Size([4, 6, 3, 3])
conv2_out - torch.Size([4, 6, 3, 3])
bn2_out - torch.Size([4, 6, 3, 3])
conv3_out - torch.Size([4, 6, 3, 3])
Y - torch.Size([4, 6, 3, 3])


In [66]:
blk

Residual(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(3, 6, kernel_size=(1, 1), stride=(2, 2))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

## 5.11.2 ResNet模型

In [84]:
net = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(64), 
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [85]:
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    if first_block:
        assert in_channels == out_channels # 第一个模块的通道数同输入通道数一致
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

In [86]:
net.add_module("resnet_block1", resnet_block(64, 64, 2, first_block=True))
net.add_module("resnet_block2", resnet_block(64, 128, 2))
net.add_module("resnet_block3", resnet_block(128, 256, 2))
net.add_module("resnet_block4", resnet_block(256, 512, 2))

In [87]:
net.add_module("global_avg_pool", d2l.GlobalAvgPool2d()) # GlobalAvgPool2d的输出: (Batch, 512, 1, 1)
net.add_module("flatten", nn.Sequential(d2l.FlattenLayer()))
net.add_module("fc", nn.Sequential(nn.Linear(512, 10)))

In [88]:
X = torch.rand((1, 1, 224, 224))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

0  output shape:	 torch.Size([1, 64, 112, 112])
1  output shape:	 torch.Size([1, 64, 112, 112])
2  output shape:	 torch.Size([1, 64, 112, 112])
3  output shape:	 torch.Size([1, 64, 56, 56])
X - torch.Size([1, 64, 56, 56])
Y - torch.Size([1, 64, 56, 56])
X - torch.Size([1, 64, 56, 56])
Y - torch.Size([1, 64, 56, 56])
resnet_block1  output shape:	 torch.Size([1, 64, 56, 56])
X - torch.Size([1, 64, 56, 56])
Y - torch.Size([1, 128, 28, 28])
X - torch.Size([1, 128, 28, 28])
Y - torch.Size([1, 128, 28, 28])
resnet_block2  output shape:	 torch.Size([1, 128, 28, 28])
X - torch.Size([1, 128, 28, 28])
Y - torch.Size([1, 256, 14, 14])
X - torch.Size([1, 256, 14, 14])
Y - torch.Size([1, 256, 14, 14])
resnet_block3  output shape:	 torch.Size([1, 256, 14, 14])
X - torch.Size([1, 256, 14, 14])
Y - torch.Size([1, 512, 7, 7])
X - torch.Size([1, 512, 7, 7])
Y - torch.Size([1, 512, 7, 7])
resnet_block4  output shape:	 torch.Size([1, 512, 7, 7])
global_avg_pool  output shape:	 torch.Size([1, 512, 1, 1])
f

In [90]:
# net

## 5.11.3 获取数据和训练模型

In [10]:
batch_size = 256
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.0015, train acc 0.853, test acc 0.885, time 31.0 sec
epoch 2, loss 0.0010, train acc 0.910, test acc 0.899, time 31.8 sec
epoch 3, loss 0.0008, train acc 0.926, test acc 0.911, time 31.6 sec
epoch 4, loss 0.0007, train acc 0.936, test acc 0.916, time 31.8 sec
epoch 5, loss 0.0006, train acc 0.944, test acc 0.926, time 31.5 sec
